# Convert Raster Data to DGGS Zones

In [1]:
import os.path

import folium
import json
import pystac_client
import shapely
from tqdm.autonotebook import tqdm

from vgrid.conversion.raster2dggs import raster2a5, raster2rhealpix, raster2h3
from vgrid.utils.io import convert_to_output_format, download_file

/tmp/ipykernel_983305/1827741114.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# [optional] extra dependencies for nicer notebook progress bars if not available
!pip install -U tqdm ipywidgets jupyterlab_widgets

## Identify the Study Area of Interest

In [3]:
with open("../manitoba_study_area/inputs/manitoba_study_area.geojson") as f:
    data = f.read()
    poly = json.loads(data)
    geom = shapely.from_geojson(data)

map = folium.Map(location=[geom.centroid.y, geom.centroid.x], zoom_start=8)
folium.GeoJson(poly).add_to(map)
map

## Find RCM ARD Data from EODMS matching AOI

In [4]:
client = pystac_client.Client.open("https://www.eodms-sgdot.nrcan-rncan.gc.ca/stac/")
result = client.search(
    collections=["rcm-ard"],
    intersects=geom,
    datetime=["2018-01-01", "2026-01-01"],
    limit=100,
)
result.matched()

75

In [5]:
item = next(result.items())
item

<Item id=12493b3f-4304-4d50-b067-c5f4a002d89d>

## Convert RCM ARD Data to DGGS

#### NOTE

⚠️ Requires patch: https://github.com/opengeoshub/vgrid/pull/57


In [6]:
output_formats = ["geojson", "parquet"]
dggs_types_res = [
    ("H3", raster2h3, [7, 8]),  # H3 L8 hexagon dimension ~= ISEA7H L9
    # ("A5", raster2a5, [9]),
    # ("rHEALPix", raster2rhealpix, [8, 9]),
]

dggs_bar = tqdm(dggs_types_res, desc="Processing DGGS Types")
for dggrs, dggrs_builder, dggrs_resolutions in dggs_bar:
    dggs_bar.set_postfix(dggrs=dggrs)
    res_bar = tqdm(dggrs_resolutions, desc="Processing Resolutions", leave=False, total=len(dggrs_resolutions))
    for resolution in res_bar:
        res_bar.set_postfix(dggrs=dggrs, resolution=resolution)
        stac_items_bar = tqdm(result.items(), desc="Processing STAC Items", leave=False, total=result.matched())
        for item in stac_items_bar:
            stac_items_bar.set_postfix(dggrs=dggrs, resolution=resolution, item=item.id)
            date = item.properties["datetime"][:10]  # YYYY-MM-DD
            for band in ["rl", "rr"]:
                src_uri = item.assets[band].href
                src_name = os.path.basename(src_uri)
                res_bar.set_postfix(dggrs=dggrs, resolution=resolution, item=src_name)

                src_dir = os.path.join("./outputs", f"manitoba_rcm_ard", "src", date, band)
                src_path = os.path.join(src_dir, src_name)
                out_dir = os.path.join("./outputs", "manitoba_rcm_ard", dggrs, f"L{resolution}", date, band)
                os.makedirs(out_dir, exist_ok=True)
                out_files = {
                    fmt: os.path.join(out_dir, f"{os.path.splitext(src_name)[0]}.{fmt}")
                    for fmt in output_formats
                }
                if all(os.path.isfile(p) for p in out_files.values()):
                    print(f"  Skipping... All output files already exist: {list(out_files.values())}")
                    continue

                print(f"Converting to {dggrs}-L{resolution} [band={band}]. Saving to [{out_dir}]...")
                if not os.path.isfile(src_path):
                    os.makedirs(src_dir, exist_ok=True)
                    download_file(src_uri, src_path)
                out_df = dggrs_builder(
                    src_path,
                    resolution=resolution,
                    output_format="gpd"  # GeoPandas Dataframe
                )
                # save results
                for fmt, out_path in out_files.items():
                    convert_to_output_format(out_df, output_format=fmt, output_name=out_path)

Processing DGGS Types:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Resolutions:   0%|          | 0/2 [00:00<?, ?it/s]

Processing STAC Items:   0%|          | 0/75 [00:00<?, ?it/s]

Converting to H3-L7 [band=rl]. Saving to [./outputs/manitoba_rcm_ard/H3/L7/2025-08-31/rl]...





Converting raster to H3:   0%|          | 0/6345 [00:00<?, ? cells/s]


Converting raster to H3:   2%|▏         | 98/6345 [00:00<00:06, 963.93 cells/s]


Converting raster to H3:   3%|▎         | 211/6345 [00:00<00:05, 1056.65 cells/s]


Converting raster to H3:   5%|▌         | 341/6345 [00:00<00:05, 1164.44 cells/s]


Converting raster to H3:   7%|▋         | 458/6345 [00:00<00:05, 1137.88 cells/s]


Converting raster to H3:   9%|▉         | 572/6345 [00:00<00:05, 1098.95 cells/s]


Converting raster to H3:  11%|█         | 683/6345 [00:00<00:05, 1045.10 cells/s]


Converting raster to H3:  12%|█▏        | 789/6345 [00:00<00:05, 939.00 cells/s] 


Converting raster to H3:  14%|█▍        | 885/6345 [00:00<00:07, 756.08 cells/s]


Converting raster to H3:  15%|█▌        | 967/6345 [00:01<00:07, 747.06 cells/s]


Converting raster to H3:  17%|█▋        | 1060/6345 [00:01<00:06, 792.12 cells/s]


Converting raster to H3:  18%|█▊        | 1147/6345 [00:01<00:06, 811.41 cells/s]


Conve

...

Output file saved as: /home/francis/dev/ogc/ogc-dggs/ogc11-dggs/manitoba-rcm-ard/./outputs/manitoba_rcm_ard/H3/L8/2025-04-02/rr/RCM2_OK3433412_PK3546765_1_SC30MCPA_20250402_000556_RR.geojson
Output file saved as: /home/francis/dev/ogc/ogc-dggs/ogc11-dggs/manitoba-rcm-ard/./outputs/manitoba_rcm_ard/H3/L8/2025-04-02/rr/RCM2_OK3433412_PK3546765_1_SC30MCPA_20250402_000556_RR.parquet
Converting to H3-L8 [band=rl]. Saving to [./outputs/manitoba_rcm_ard/H3/L8/2025-04-01/rl]...





Converting raster to H3:   0%|          | 0/38787 [00:00<?, ? cells/s]


Converting raster to H3:   1%|          | 451/38787 [00:00<00:08, 4507.48 cells/s]


Converting raster to H3:   2%|▏         | 902/38787 [00:00<00:09, 3994.34 cells/s]


Converting raster to H3:   3%|▎         | 1306/38787 [00:00<00:09, 3950.07 cells/s]


Converting raster to H3:   4%|▍         | 1731/38787 [00:00<00:09, 4062.33 cells/s]


Converting raster to H3:   6%|▌         | 2175/38787 [00:00<00:08, 4193.97 cells/s]


Converting raster to H3:   7%|▋         | 2608/38787 [00:00<00:08, 4234.77 cells/s]


Converting raster to H3:   8%|▊         | 3039/38787 [00:00<00:08, 4256.98 cells/s]


Converting raster to H3:   9%|▉         | 3486/38787 [00:00<00:08, 4322.17 cells/s]


Converting raster to H3:  10%|█         | 3919/38787 [00:00<00:08, 4294.73 cells/s]


Converting raster to H3:  11%|█         | 4355/38787 [00:01<00:07, 4313.64 cells/s]


Converting raster to H3:  12%|█▏        | 4827/38787 [00:01<00:07,

Output file saved as: /home/francis/dev/ogc/ogc-dggs/ogc11-dggs/manitoba-rcm-ard/./outputs/manitoba_rcm_ard/H3/L8/2025-04-01/rl/RCM3_OK3433412_PK3545895_4_SC30MCPD_20250401_123803_RL.geojson
Output file saved as: /home/francis/dev/ogc/ogc-dggs/ogc11-dggs/manitoba-rcm-ard/./outputs/manitoba_rcm_ard/H3/L8/2025-04-01/rl/RCM3_OK3433412_PK3545895_4_SC30MCPD_20250401_123803_RL.parquet
Converting to H3-L8 [band=rr]. Saving to [./outputs/manitoba_rcm_ard/H3/L8/2025-04-01/rr]...





Converting raster to H3:   0%|          | 0/38787 [00:00<?, ? cells/s]


Converting raster to H3:   1%|          | 390/38787 [00:00<00:09, 3898.77 cells/s]


Converting raster to H3:   2%|▏         | 780/38787 [00:00<00:10, 3679.49 cells/s]


Converting raster to H3:   3%|▎         | 1149/38787 [00:00<00:10, 3548.99 cells/s]


Converting raster to H3:   4%|▍         | 1505/38787 [00:00<00:10, 3401.42 cells/s]


Converting raster to H3:   5%|▍         | 1846/38787 [00:00<00:10, 3384.30 cells/s]


Converting raster to H3:   6%|▌         | 2185/38787 [00:00<00:12, 2963.79 cells/s]


Converting raster to H3:   7%|▋         | 2553/38787 [00:00<00:11, 3166.72 cells/s]


Converting raster to H3:   8%|▊         | 2917/38787 [00:00<00:10, 3304.20 cells/s]


Converting raster to H3:   9%|▊         | 3308/38787 [00:00<00:10, 3482.45 cells/s]


Converting raster to H3:   9%|▉         | 3684/38787 [00:01<00:09, 3563.30 cells/s]


Converting raster to H3:  11%|█         | 4095/38787 [00:01<00:09,

Output file saved as: /home/francis/dev/ogc/ogc-dggs/ogc11-dggs/manitoba-rcm-ard/./outputs/manitoba_rcm_ard/H3/L8/2025-04-01/rr/RCM3_OK3433412_PK3545895_4_SC30MCPD_20250401_123803_RR.geojson
Output file saved as: /home/francis/dev/ogc/ogc-dggs/ogc11-dggs/manitoba-rcm-ard/./outputs/manitoba_rcm_ard/H3/L8/2025-04-01/rr/RCM3_OK3433412_PK3545895_4_SC30MCPD_20250401_123803_RR.parquet
